# 样例介绍

功能：使用cartoonGAN模型对输入图片进行卡通化处理。  
样例输入：原始图片jpg文件。  
样例输出：卡通图像。

# 前期准备

基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：

1. 获取模型和测试数据。我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，将模型和测试数据下载到本地，并放到本样例目录下。

2. 模型转换。利用atc工具将原始模型转换为om模型，转换命令如下：  
    ```shell
    atc --output_type=FP32 --input_shape="train_real_A:1,256,256,3" --input_format=NHWC --output="./cartoonization" --soc_version=Ascend310B1 --insert_op_conf=insert_op.cfg --framework=3 --save_original_model=false --model="./cartoonization.pb" --precision_mode=allow_fp32_to_fp16
    ```

    其中各个参数具体含义如下：
    * --output_type：指定网络输出数据类型。
    * --input_shape：模型的输入节点名称和形状。
    * --input_format：输入Tensor的内存排列方式。
    * --output：输出的模型文件路径。
    * --soc_version：昇腾AI处理器型号，此处为"Ascend310B1"。
    * --insert_op_conf：插入算子的配置文件路径与文件名，例如aipp预处理算子。
    * --framework：原始框架类型,  0: Caffe, 1: MindSpore, 3: TensorFlow, 5: ONNX。
    * --save_original_model：转换后是否保留原始模型文件。
    * --model：原始模型文件路径。
    * --precision_mode：选择算子精度模式。

# 模型推理实现

### 1. 导入三方库

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import acl

import acllite_utils as utils
import constants as const
from acllite_imageproc import AclLiteImageProc
from acllite_model import AclLiteModel
from acllite_image import AclLiteImage
from acllite_resource import resource_list

### 2. 定义acllite资源初始化与去初始化类

In [ ]:
class AclLiteResource:
    """
    AclLiteResource
    """
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """
        init resource
        """
        print("init resource stage:")
        ret = acl.init()

        ret = acl.rt.set_device(self.device_id)
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id)
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream()
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode()
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream)

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context)

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id)
        print("Release acl resource success")

### 3. 定义卡通化类，包含前处理、推理、后处理等操作

In [ ]:
class Cartoonization(object):
    """
    class for Cartoonization
    """
    def __init__(self, model_path, model_width, model_height):
        self._model_path = model_path
        self._model_width = model_width
        self._model_height = model_height
        self.device_id = 0
        self._dvpp = None
        self._model = None

    def init(self):
        """
        Initialize
        """
        # Initialize dvpp
        self._dvpp = AclLiteImageProc()

        # Load model
        self._model = AclLiteModel(self._model_path)

        return const.SUCCESS
 
    @utils.display_time
    def pre_process(self, image):
        """
        image preprocess
        """
        image_dvpp = image.copy_to_dvpp()
        yuv_image = self._dvpp.jpegd(image_dvpp)  # 将jpg格式图片转为yuv格式
        crop_and_paste_image = self._dvpp.crop_and_paste_get_roi(yuv_image, image.width, image.height, \
                                self._model_width, self._model_height)  # 裁剪图片
        return crop_and_paste_image

    @utils.display_time
    def inference(self, resized_image):
        """
        model inference
        """
        return self._model.execute(resized_image)

    @utils.display_time
    def post_process(self, infer_output, image_file, origin_image):
        """
        post process
        """
        data = ((np.squeeze(infer_output[0]) + 1) * 127.5)  # 将像素值转换到0-255之间
        img = cv2.cvtColor(data, cv2.COLOR_RGB2BGR) 
        img = cv2.resize(img, (origin_image.width, origin_image.height))  # 缩放到原始图片大小
        
        # plot result
        plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        img = img[:,:,[2,1,0]]
        plt.imshow(img/255)

### 4. 构造主函数，串联整个代码逻辑

In [ ]:
def main():
    MODEL_PATH = "cartoonization.om"
    MODEL_WIDTH = 256
    MODEL_HEIGHT = 256

    acl_resource = AclLiteResource()  # 初始化acl资源
    acl_resource.init()
    
    # instantiation Cartoonization object
    cartoonization = Cartoonization(MODEL_PATH, MODEL_WIDTH, MODEL_HEIGHT)  # 构造模型对象
    
    # init
    ret = cartoonization.init()  # 初始化模型类变量
    utils.check_ret("Cartoonization.init ", ret)  
    

    image_file = 'img.jpg'
    # read image
    image = AclLiteImage(image_file)  # 构造 AclLiteImage 对象，方便利用 dvpp 进行前处理
    
    print('===================')
    print(image)
    # preprocess
    crop_and_paste_image = cartoonization.pre_process(image)  # 前处理
    # inference
    result = cartoonization.inference([crop_and_paste_image, ])  # 推理
    # postprocess
    cartoonization.post_process(result, image_file, image)  # 后处理

### 5. 运行
运行完成后，会显示推理后的图片，如下所示。

In [ ]:
main()

其中，除了acl相关资源初始化和释放的信息外，“in pre_process, use time”表示前处理耗时，“in inference, use time”表示推理耗时，“in post_process, use time”表示后处理耗时，单位都为秒。

# 样例总结

我们来回顾一下以上代码，可以包括以下几个步骤：

1. 初始化acl资源：在调用acl相关资源时，必须先初始化AscendCL，否则可能会导致后续系统内部资源初始化出错。  
2. 对图片进行前处理：在此样例中，我们首先根据图片路径，构造AclLiteImage类型的数据，再利用.jpegd和.crop_and_paste_get_roi转换图片格式、裁剪图片。注意由于模型输入是yuv格式，所以我们利用了AclLiteImageProc.jpegd将图片转为yuv，使得模型正常推理。  
3. 推理：利用AclLiteModel.execute接口对图片进行推理。  
4. 对推理结果进行后处理：包括两个步骤，即转换像素值值域以及将图片缩放到原图大小。  
5. 可视化图片：利用plt将结果画出。